In [1]:
import pandas as pd
import os
from estnltk.converters.conll.conll_importer import conll_to_text
from estnltk_core.layer_operations import split_by_sentences
import estnltk
from estnltk import Text
from tqdm import tqdm

In [ ]:
# mis on fraasi peasõna ja verb

In [2]:
def get_peasona_and_verb(input_file, sent, phrase):
    text_obj = conll_to_text( input_file, syntax_layer='stanza_syntax' )
    texts2 = split_by_sentences(text=text_obj,
                                   layers_to_keep=list(text_obj.layers),
                                   trim_overlapping=True
                                   )
    # target text object for the sentence
    target = None
    target_idx = None
    for i,txt in enumerate(texts2):
        #clean_txt = clean(txt.text)
        if txt.text == sent:
            target = txt
            target_idx = i
    if target_idx is None:
        for i,txt in enumerate(texts2):
            clean_txt = clean(txt.text)
            clean_sent = clean(sent)
            if clean_txt == clean_sent:
                target = txt
                target_idx = i
    
    # target layer
    target_layer = texts2[target_idx]["stanza_syntax"]
    #display(target_layer)
    
    # words in the removed phrase
    phrase_words = {}
    for word in phrase.split(" "):
        phrase_words[word] = None
    # and corresponding indexes for the phrase words
    for idx, span in enumerate(target_layer):
        if span.text in phrase_words.keys():
            phrase_words[span.text] = idx
    # heads for the phrase words
    phrase_words_heads = {}
    for word in phrase_words.keys():
        if phrase_words[word] is not None: 
            # "Stuart Little 79 398" numbrid on ühe sõnana aga kuna siin on need tühiku alusel lahku löödud
            # siis jääb nende asemel None ja võimalik, et ei leita verbi
            phrase_words_heads[word] = target_layer[phrase_words[word]].head   
    # word with the smallest head is the main word in phrase
    peasona = min(phrase_words_heads, key=phrase_words_heads.get)
    peasona_idx = phrase_words[peasona]
    peasonahead = phrase_words_heads[peasona]
    
    ## verbi otsimine
    phrase_verb = None
    isverb = False
    if peasonahead != 0:
        # peasona on verb
        if target_layer[peasona_idx].xpostag == "V":
            #print("peasona on verb")
            phrase_verb = peasona_idx
            isverb = True

        # peasona head on verb
        elif target_layer[peasonahead-1].xpostag == "V":
            phrase_verb = peasonahead
            isverb = True
            #print("head on verb", phrase_verb)

        # peasona head ei ole verb, tuleb hakata ülespoole liikuma
        elif target_layer[peasonahead-1].xpostag != "V":
            new_head = peasonahead
            phrase_verb = None
            while not isverb:
                #print("peasõna head", new_head, "ehk ", target_layer[new_head-1])
                if target_layer[new_head-1].xpostag == "V":
                    isverb = True
                    phrase_verb = new_head
                else:
                    new_head = target_layer[new_head-1].head
                    if new_head == 0 and isverb==False: # kui jõudsime rootini aga verbi ei ole
                        break

    # fraasi peasõna ei ole verbi alluv, võta siis esimene verb
    if not isverb or peasonahead == 0:
        for span in target_layer:
            if span.upostag == "VERB" or span.xpostag == "V":
                phrase_verb = span.id
                break
    
    verb = None
    if phrase_verb is not None:
        verb = target_layer[phrase_verb-1].text
    
    
    # return peasõna, peasõna index, peasõna parent ehk head, verb, verbi id
    return peasona, peasona_idx, peasonahead, verb, phrase_verb

In [3]:
deprels = ["acl", "advcl", "advmod", "appos", "det", "nmod", "nummod", "obl"] # "discourse", "vocative"

root = r".../UDpuupank/UD2_11_udreposse"

In [4]:
deprel = deprels[3] 
# nummod [6] on probleemne 
# phrase_words_heads[word] = target_layer
# TypeError: index not supported: None

In [5]:
print(deprel)
data = pd.read_csv(f"data/{deprel}_benchmark_1000.csv", sep=";", encoding="utf-8")
free_sent_data = []
for i in tqdm(range(len(data))):
    #if i == 441:
    sent = data.iloc[i]["sentence"]
    phrase = data.iloc[i]["removed"]
    file = data.iloc[i]["fpath"]
    #print(sent)
    input_file = os.path.join(root, file) 
    free_sent_data.append((get_peasona_and_verb(input_file, sent, phrase)))


appos


100%|█████████████████████████████████████| 1000/1000 [1:24:29<00:00,  5.07s/it]


In [6]:
data["sentence_analysis_data"] = free_sent_data
free_peasonad = []
free_verbs = []

for data2 in free_sent_data:
    free_peasonad.append(data2[0])
    free_verbs.append(data2[3])

data["peasona"] = free_peasonad
data["verb"] = free_verbs

data.to_csv(f"data/{deprel}_benchmark_1000_peasona_verb.csv", index=False, sep=";", encoding='utf-8')